In [6]:
import os
import open3d as o3d
import numpy as np
import cv2
from matplotlib import pyplot as plt
import random

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [10]:
# Paths to the folders
depth_folder = "car_challenge/depth"
rgb_folder = "car_challenge/rgb"
output_folder = "car_challenge/point_clouds"
icp_output_folder = "car_challenge/icp_point_clouds"

# Create output folders if they don't exist
os.makedirs(output_folder, exist_ok=True)
os.makedirs(icp_output_folder, exist_ok=True)

# Get sorted lists of file names
depth_images = sorted(os.listdir(depth_folder))
rgb_images = sorted(os.listdir(rgb_folder))

# Lists to hold matching pairs
matching_pairs = []

# Use only images that have a match
min_length = min(len(depth_images), len(rgb_images))
for i in range(min_length):
    depth_name, rgb_name = os.path.splitext(depth_images[i])[0], os.path.splitext(rgb_images[i])[0]
    if depth_name == rgb_name:
        matching_pairs.append((os.path.join(depth_folder, depth_images[i]), os.path.join(rgb_folder, rgb_images[i])))
    else:
        print(f"Mismatch found: {depth_images[i]} (depth) vs {rgb_images[i]} (RGB)")

if matching_pairs:
    print(f"Number of matching pairs: {len(matching_pairs)}")
else:
    print("No matching pairs found.")


Number of matching pairs: 1722


In [41]:
# Function to create point cloud from images
def create_point_cloud(depth_image, rgb_image, solid_color=None):
    h, w = depth_image.shape
    fx, fy = 525.0, 525.0  # Focal length (example values, adjust accordingly)
    cx, cy = w / 2, h / 2  # Principal point

    points = []
    colors = []

    for v in range(h):
        for u in range(w):
            z = depth_image[v, u] / 1000.0  # Convert depth to meters
            if z == 0:
                continue
            x = (u - cx) * z / fx
            y = (v - cy) * z / fy
            points.append((x, y, z))
            if solid_color is not None:
                colors.append(solid_color)
            else:
                colors.append(rgb_image[v, u] / 255.0)

    # Create Open3D point cloud
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(np.array(points))
    point_cloud.colors = o3d.utility.Vector3dVector(np.array(colors))

    return point_cloud

# Function to visualize point clouds
def visualize_point_clouds(point_cloud_ids, solid_colors=False):
    point_clouds = []
    for point_cloud_id in point_cloud_ids:
        point_cloud_path = os.path.join(output_folder, f"point_cloud_{point_cloud_id:04d}.ply")
        if not os.path.exists(point_cloud_path):
            print(f"Point cloud file not found: {point_cloud_path}")
            continue

        # Load point cloud
        point_cloud = o3d.io.read_point_cloud(point_cloud_path)
        if solid_colors:
            # Generate a random color for the point cloud
            color = [random.random(), random.random(), random.random()]
            point_cloud.paint_uniform_color(color)

        print(f"Point cloud {point_cloud_id} has {len(point_cloud.points)} points.")
        point_clouds.append(point_cloud)

    # Visualize point clouds
    if point_clouds:
        o3d.visualization.draw_geometries(point_clouds, window_name="Point Cloud Visualization", width=800, height=600)


# Function to generate point clouds and save them to files with skipping capability
def generate_and_save_point_clouds(skip=1):
    skipped_pairs = matching_pairs[::skip]
    print(f"Processing {len(skipped_pairs)} point clouds out of {len(matching_pairs)} with skip={skip}.")
    for idx, (depth_path, rgb_path) in enumerate(skipped_pairs):
        # Load images
        depth_image = cv2.imread(depth_path, cv2.IMREAD_UNCHANGED)
        rgb_image = cv2.imread(rgb_path, cv2.IMREAD_COLOR)

        # Create point cloud
        point_cloud = create_point_cloud(depth_image, rgb_image)

        # Downsample point cloud if necessary
        num_points = len(point_cloud.points)
        iteration = 0
        while num_points > 20000:
            iteration += 1
            voxel_size = 0.01 + (iteration * 0.01)  # Increase voxel size gradually to ensure reduction
            point_cloud = point_cloud.voxel_down_sample(voxel_size=voxel_size)
            num_points = len(point_cloud.points)
            print(f"Iteration {iteration}: Downsampled point cloud to {num_points} points with voxel size {voxel_size}.")

        # Save point cloud to output folder
        output_path = os.path.join(output_folder, f"point_cloud_{idx * skip:04d}.ply")
        o3d.io.write_point_cloud(output_path, point_cloud)
        print(f"Saved point cloud: {output_path}")
        print(f"Point cloud {idx * skip} has {len(point_cloud.points)} points.")


In [76]:
# Generate point clouds
generate_and_save_point_clouds(skip=5)

Processing 345 point clouds out of 1722 with skip=5.
Iteration 1: Downsampled point cloud to 27223 points with voxel size 0.02.
Iteration 2: Downsampled point cloud to 12113 points with voxel size 0.03.
Saved point cloud: car_challenge/point_clouds\point_cloud_0000.ply
Point cloud 0 has 12113 points.
Iteration 1: Downsampled point cloud to 27327 points with voxel size 0.02.
Iteration 2: Downsampled point cloud to 12092 points with voxel size 0.03.
Saved point cloud: car_challenge/point_clouds\point_cloud_0005.ply
Point cloud 5 has 12092 points.
Iteration 1: Downsampled point cloud to 26928 points with voxel size 0.02.
Iteration 2: Downsampled point cloud to 11895 points with voxel size 0.03.
Saved point cloud: car_challenge/point_clouds\point_cloud_0010.ply
Point cloud 10 has 11895 points.
Iteration 1: Downsampled point cloud to 26524 points with voxel size 0.02.
Iteration 2: Downsampled point cloud to 11694 points with voxel size 0.03.
Saved point cloud: car_challenge/point_clouds\poi

In [75]:
visualize_point_clouds(point_cloud_ids=[1000, 400], solid_colors=True)

Point cloud 1000 has 14593 points.
Point cloud 400 has 16437 points.


In [39]:
import os
import numpy as np
import open3d as o3d

# Function to perform pairwise registration using consecutive point clouds
def pairwise_registration_consecutive(skip=1, fitness_threshold=0.7):
    # Paths to the folders
    output_folder = "car_challenge/point_clouds"
    transformation_folder = "car_challenge/transformations"

    # Create folder for saving transformations
    os.makedirs(transformation_folder, exist_ok=True)

    point_cloud_files = sorted(os.listdir(output_folder))[::skip]
    point_cloud_files = [f for f in point_cloud_files if f.endswith('.ply')]
    total_point_clouds = len(point_cloud_files)

    if total_point_clouds < 2:
        print("Not enough point clouds to perform registration.")
        return [], []

    # Initialize transformations list
    transformations = [np.eye(4)]  # The first point cloud is at the origin
    valid_indices = [0]  # Index of point clouds that are successfully registered

    for i in range(1, total_point_clouds):
        source_idx = i - 1  # Use the previous point cloud as the source
        target_idx = i      # Current point cloud as the target

        # Check if transformation file already exists
        transformation_path = os.path.join(transformation_folder, f"transformation_{target_idx:04d}.npy")
        if os.path.exists(transformation_path):
            print(f"Skipping point cloud {target_idx}: Transformation already exists.")
            # Load and append the transformation matrix to the list
            transformation = np.load(transformation_path)
            transformations.append(transformation)
            valid_indices.append(target_idx)
            continue

        source_path = os.path.join(output_folder, point_cloud_files[source_idx])
        target_path = os.path.join(output_folder, point_cloud_files[target_idx])

        source_pcd = o3d.io.read_point_cloud(source_path)
        target_pcd = o3d.io.read_point_cloud(target_path)

        print(f"Registering point cloud {source_idx} to point cloud {target_idx}")

        # Downsample point clouds
        voxel_size = 0.005  # Adjusted voxel size for balance
        source_down = source_pcd.voxel_down_sample(voxel_size)
        target_down = target_pcd.voxel_down_sample(voxel_size)

        # Estimate normals
        radius_normal = 2  # Set a fixed radius
        max_nn_normals = 100  # Increased max_nn
        source_down.estimate_normals(
            o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=max_nn_normals))
        target_down.estimate_normals(
            o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=max_nn_normals))

        # Compute FPFH features
        radius_feature = 0.1  # Set a fixed radius
        max_nn_fpfh = 200  # Increased max_nn
        source_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
            source_down,
            o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=max_nn_fpfh))
        target_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
            target_down,
            o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=max_nn_fpfh))

        # Perform RANSAC-based global registration
        distance_threshold = 0.05  # Set a fixed threshold
        result_ransac = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
            source_down, target_down, source_fpfh, target_fpfh, mutual_filter=True,
            max_correspondence_distance=distance_threshold,
            estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
            ransac_n=4,
            checkers=[
                o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
                o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
            ],
            criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(
                max_iteration=2000000, confidence=0.99999)
        )

        # Refine registration with ICP
        threshold_icp = 0.02
        # Re-estimate normals on full point clouds
        radius_normal_full = 0.05
        source_pcd.estimate_normals(
            o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal_full, max_nn=max_nn_normals))
        target_pcd.estimate_normals(
            o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal_full, max_nn=max_nn_normals))

        result_icp = o3d.pipelines.registration.registration_icp(
            source_pcd, target_pcd, threshold_icp, result_ransac.transformation,
            o3d.pipelines.registration.TransformationEstimationPointToPlane(),
            criteria=o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=10000)
        )

        print(f"Pair {source_idx}-{target_idx} registration result: Fitness - {result_icp.fitness}, Inlier RMSE - {result_icp.inlier_rmse}")

        # Check fitness and decide whether to accept the registration
        if result_icp.fitness >= fitness_threshold:
            # Accumulate transformation
            delta_transformation = result_icp.transformation  # Transformation from source to target
            transformation = delta_transformation @ transformations[source_idx]
            transformations.append(transformation)
            valid_indices.append(target_idx)

            # Save the transformation matrix to file
            np.save(transformation_path, transformation)
            print(f"Transformation saved: {transformation_path}")
        else:
            print(f"Pair {source_idx}-{target_idx} rejected due to low fitness {result_icp.fitness}")
            # Keep the last transformation
            transformations.append(transformations[source_idx])

    return transformations, valid_indices


In [42]:
transformations, valid_indices = pairwise_registration_consecutive(skip=1, fitness_threshold=0.8)

Registering point cloud 0 to point cloud 1
Pair 0-1 registration result: Fitness - 0.8015355403285727, Inlier RMSE - 0.011642700344936618
Transformation saved: car_challenge/transformations\transformation_0001.npy
Registering point cloud 1 to point cloud 2
Pair 1-2 registration result: Fitness - 0.8242639761826, Inlier RMSE - 0.012384546366748562
Transformation saved: car_challenge/transformations\transformation_0002.npy
Registering point cloud 2 to point cloud 3
Pair 2-3 registration result: Fitness - 0.8129466162253047, Inlier RMSE - 0.013614344979127533
Transformation saved: car_challenge/transformations\transformation_0003.npy
Registering point cloud 3 to point cloud 4
Pair 3-4 registration result: Fitness - 0.8155464340687532, Inlier RMSE - 0.01300086409364495
Transformation saved: car_challenge/transformations\transformation_0004.npy
Registering point cloud 4 to point cloud 5
Pair 4-5 registration result: Fitness - 0.7770379854886897, Inlier RMSE - 0.013198229939543723
Pair 4-5 r

KeyboardInterrupt: 

In [10]:
import os
import open3d as o3d
import numpy as np

# Function to merge point clouds using saved transformations
def merge_point_clouds_filtered(skip=1):
    # Paths to the folders
    output_folder = "car_challenge/point_clouds"
    icp_output_folder = "car_challenge/icp_point_clouds"
    transformation_folder = "car_challenge/transformations"

    # Ensure the output folder exists
    os.makedirs(icp_output_folder, exist_ok=True)

    # Get the list of point cloud files
    point_cloud_files = sorted(os.listdir(output_folder))[::skip]
    point_cloud_files = [f for f in point_cloud_files if f.endswith('.ply')]

    merged_pcd = o3d.geometry.PointCloud()

    for idx, pcd_file in enumerate(point_cloud_files):
        pcd_path = os.path.join(output_folder, pcd_file)
        pcd = o3d.io.read_point_cloud(pcd_path)
        
        # Load the corresponding transformation
        transformation_path = os.path.join(transformation_folder, f"transformation_{idx:04d}.npy")
        if os.path.exists(transformation_path):
            transformation = np.load(transformation_path)
            pcd.transform(transformation)
            merged_pcd += pcd
            print(f"Merged point cloud {idx} (file: {pcd_file}) with transformation {transformation_path}")
        else:
            print(f"Transformation file not found for point cloud {idx} (file: {pcd_file}). Skipping.")

    # Optionally, downsample the merged point cloud to reduce size
    voxel_size = 0.02  # Adjust as needed
    merged_pcd = merged_pcd.voxel_down_sample(voxel_size)

    # Save the merged point cloud
    merged_pcd_path = os.path.join(icp_output_folder, "merged_point_cloud_filtered.ply")
    o3d.io.write_point_cloud(merged_pcd_path, merged_pcd)
    print(f"Saved merged point cloud to {merged_pcd_path}")

    # Visualize the merged point cloud
    o3d.visualization.draw_geometries([merged_pcd], window_name="Filtered Merged Point Cloud", width=800, height=600)


In [43]:
# Merge the point clouds using the transformations and valid indices
merge_point_clouds_filtered(skip=1)

Transformation file not found for point cloud 0 (file: point_cloud_0000.ply). Skipping.
Merged point cloud 1 (file: point_cloud_0005.ply) with transformation car_challenge/transformations\transformation_0001.npy
Merged point cloud 2 (file: point_cloud_0010.ply) with transformation car_challenge/transformations\transformation_0002.npy
Merged point cloud 3 (file: point_cloud_0015.ply) with transformation car_challenge/transformations\transformation_0003.npy
Merged point cloud 4 (file: point_cloud_0020.ply) with transformation car_challenge/transformations\transformation_0004.npy
Transformation file not found for point cloud 5 (file: point_cloud_0025.ply). Skipping.
Transformation file not found for point cloud 6 (file: point_cloud_0030.ply). Skipping.
Transformation file not found for point cloud 7 (file: point_cloud_0035.ply). Skipping.
Merged point cloud 8 (file: point_cloud_0040.ply) with transformation car_challenge/transformations\transformation_0008.npy
Transformation file not foun

In [87]:
def visualize_merged_point_cloud():
    merged_pcd_path = os.path.join(icp_output_folder, "merged_point_cloud.ply")
    if not os.path.exists(merged_pcd_path):
        print("Merged point cloud not found.")
        return
    merged_pcd = o3d.io.read_point_cloud(merged_pcd_path)
    o3d.visualization.draw_geometries([merged_pcd], window_name="Merged Point Cloud", width=800, height=600)

visualize_merged_point_cloud()


[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
